## universal functions
---

as funções universais apresentam uma forma de fazer operações diretamente com os arrays do numpy sem precisar passar com um código nativo do python, acelerando o tempo de compilação do código.

este processo é chamado de vetorização (*vectorization*) e deve ser usado para substituir loops, principalmente, naquelas situações em que o array cresce ao longo do processo.

um exemplo:

In [1]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

observe quão lento é este código...

In [2]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

2.5 s ± 701 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


... quando comparado utilizando ufunc:

In [3]:
%timeit (1.0 / big_array)

3.11 ms ± 647 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


observe que isto pode ser usado com todo tipo de função numpy:

In [4]:
np.arange(5) / np.arange(1, 6)

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

In [5]:
x = np.arange(9).reshape((3, 3))
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]])

#### operadores
---

são usados os operadores nativos do python para cálculos matemáticos: 

In [6]:
array = np.random.randint(1, 30, 5)
sa = array + 5
suba = array - 5
ma = array * 5
da = array / 5
ea = array**5
moda = array%5
print(f'{array}\nsoma: {sa}\nsubtração: {suba}\nmultiplicação: {ma}\ndivisão: {da}\nexponenciação: {ea}\nmódulo: {moda}')

[19 22 28  4 29]
soma: [24 27 33  9 34]
subtração: [14 17 23 -1 24]
multiplicação: [ 95 110 140  20 145]
divisão: [3.8 4.4 5.6 0.8 5.8]
exponenciação: [ 2476099  5153632 17210368     1024 20511149]
módulo: [4 2 3 4 4]


bem como,

In [7]:
print(f'{-array}\n{abs(array)}')

[-19 -22 -28  -4 -29]
[19 22 28  4 29]


todas essas operações podem ser substituídas por funções do numpy:


operação|função
---|---
+|np.add()
-|np.subtract()
-|np.negative()
\*|np.multiply()
/|np.devide()
//|np.floor_devide()
\*\*|np.power()
%|np.mod()
abs|np.absolute() ou np.abs()

por exemplo:

In [8]:
print(f'{np.negative(array)}\n{np.add(array, 5)}')

[-19 -22 -28  -4 -29]
[24 27 33  9 34]


há muitos métodos que complementam estas funções, refinando seu uso:

np.função.método|uso
---|---
.reduce()|itera o array e calcula seu valor
.accumulate()|itera o array, calcula seu valor e mostra os passos
.outer()|faz a operação entre duas arrays

operações mais complexas podem ser feitas também:

In [9]:
vec = np.random.randint(1, 10, (4, 5))
op = (1/2)*vec**2 - 2*vec + 4
print(f'{array}\n--operação--\n{op}')

[19 22 28  4 29]
--operação--
[[ 2.  14.5 20.   2.5  4. ]
 [ 2.5 20.  20.  14.5 26.5]
 [ 2.5  2.5  4.   4.   6.5]
 [26.5 14.5 10.  10.  14.5]]


#### argumento *out*
---

é usado quando é preciso criar muitas arrays sem que estas sejam necessárias futuramente no código. por isso, é possível criar uma array "vazia" para que o conteúdo desta seja mudado pelas informações que o usuário deseja no momento, e isto é possível usando o argumento `out=<array>`

In [10]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)
y

array([ 0., 10., 20., 30., 40.])

observe: `y` é um array coringa e seu conteúdo original não me importa. o que me importa é que `y` já existe e, com isso, não precisarei gastar mais espaço de memória quando precisar de outra array temporária.

In [11]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)

[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]


atenção que o tamanho de `y` tem que "encaixar direitinho" na operação que está sendo realizada.

#### broadcasting
---

como é possível observar, fazer operações com arrays é simples, como, por exemplo, somar todos os valores de um array pelo valor 5:

In [12]:
ar = np.arange(3)
ar

array([0, 1, 2])

In [24]:
ar + 5 # ou np.add(ar, 5)

array([5, 6, 7])

isto é chamado de broadcasting, ou seja, o valor 5, teoricamente falando, se transforma em um array de mesmo tamanho que a array `ar` e, assim, cada índice de um array se soma com o do outro: [a1, a2, a3] + [b1, b2, b3] = [a1 + b1, a2 + b2, a3 + b3].

devido ao broadcasting, com numpy, é possível fazer operações com arrays de diferentes tamanhos

In [16]:
ar1 = np.array([1, 2, 3])
ar2 = np.array([[1], [2], [3]])
print(f'{ar1}\n{ar2}')

[1 2 3]
[[1]
 [2]
 [3]]


In [17]:
ar1 + ar2

array([[2, 3, 4],
       [3, 4, 5],
       [4, 5, 6]])

explicando: o `ar1` se trasformou na seguinte matriz:

In [21]:
ar1_bc = np.array([[1, 2, 3]]*3)
ar1_bc

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])

e o `ar2` se transformou na matriz:

In [23]:
ar2_bc = np.array([[1]*3, [2]*3, [3]*3])
ar2_bc

array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]])

em seguida, elas se somaram, sempre em correspondência com seus índices.

para que o broadcasting ocorra, é necessário observar as regras:
1. se dois arrays diferem em seus número de dimensões, o formato daquele de menor dimensão é preenchido com o valor 1 (um) a partir do lado esquerdo;
2. se os formatos dos arrays não combinam em nenhuma dimensão, o array com formato igual a 1 (um) em sua dimensão é esticada para combinar com o outro;
3. se, em qualquer dimensão, os tamanhos dos arrays não combinarem nem nenhum deles têm dimensão igual a 1 (um), uma exceção ocorre.

observe que o broadcasting ocorre com todas as ufuncs, não só com a soma:

In [25]:
ar1 * ar2

array([[1, 2, 3],
       [2, 4, 6],
       [3, 6, 9]])

In [27]:
np.log(np.exp(ar1) / np.exp(ar2))

array([[ 0.,  1.,  2.],
       [-1.,  0.,  1.],
       [-2., -1.,  0.]])

observe, também, que o broadcasting funciona entre arrays, arrays e números, etc.